In [22]:
from keras.applications.xception import Xception 
import keras
from keras.preprocessing import image
from keras.applications.xception import preprocess_input
from keras.models import Model
from sklearn.model_selection import KFold as KF
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from pylab import rcParams
rcParams['figure.figsize'] = 20, 10
import os,cv2
%matplotlib inline

In [24]:
base_model = Xception(weights='imagenet',include_top=False)
# 获取各层的输出：
layer_outputs = [layer.output for layer in base_model.layers[2:20]]
# 获取各层的名称：
layer_names = []
for layer in base_model.layers[2:20]:
    layer_names.append(layer.name)
print(layer_names)

83689472/83683744 [==============================] - 3s 0us/step
['block1_conv1_bn', 'block1_conv1_act', 'block1_conv2', 'block1_conv2_bn', 'block1_conv2_act', 'block2_sepconv1', 'block2_sepconv1_bn', 'block2_sepconv2_act', 'block2_sepconv2', 'block2_sepconv2_bn', 'conv2d_1', 'block2_pool', 'batch_normalization_1', 'add_1', 'block3_sepconv1_act', 'block3_sepconv1', 'block3_sepconv1_bn', 'block3_sepconv2_act']


In [25]:
model = Model(inputs=base_model.input, outputs=layer_outputs)

In [26]:
def conj_martix(martix):
    result=[]
    width=martix.shape[0]
    height=martix.shape[1]
    nums=martix.shape[2]
    for num in range(nums):
        result.append(martix[:,:,num].flatten())
    return result
from numpy import nanmean
def downsample(myarr,factor,estimator=nanmean):
    """
    Downsample a 2D array by averaging over *factor* pixels in each axis.
    Crops upper edge if the shape is not a multiple of factor.
    This code is pure np and should be fast.
    keywords:
        estimator - default to mean.  You can downsample by summing or
            something else if you want a different estimator
            (e.g., downsampling error: you want to sum & divide by sqrt(n))
    """
    ys,xs = myarr.shape
    crarr = myarr[:ys-(ys % int(factor)),:xs-(xs % int(factor))]
    dsarr = estimator( np.concatenate([[crarr[i::factor,j::factor] 
        for i in range(factor)] 
        for j in range(factor)]), axis=0)
    return dsarr

def upper_gram(metrix):
    features=[]
    gram_len=metrix.shape[1]
    for row in range(gram_len):
        for clo in range(gram_len):
            clos=clo+row
            if(clos>gram_len-1):
                break
            features.append((metrix[row][row+clo]))
    return np.array(features)

def get_features(origin_features,layers):
# def get_features(origin_features,layers):
    features_mertix=[]
    for index in range(origin_features.shape[0]):
        cnn_featues=model.predict(origin_features[index:index+1,:,:,:])
        layer_features=cnn_featues[layers][0]#获取第三层的输出
        conj_mertix=np.array(conj_martix(layer_features))#拼接矩阵
#         down_sample_features=downsample(conj_mertix,downsample_ratio)#降采样
        gram_metrix=np.dot(conj_mertix, conj_mertix.T)#计算 gram 矩阵`
        feature=upper_gram(gram_metrix)#取 gram 矩阵的上半区
        features_mertix.append(feature)
    return np.array(features_mertix)

In [27]:
data_path = '../../jaffe/'
data_dir_list = os.listdir(data_path)
img_data_list=[]

for dataset in data_dir_list:
    img_list=os.listdir(data_path+'/'+ dataset)
    print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
    for img in img_list:
        input_img=cv2.imread(data_path + '/'+ dataset + '/'+ img )
        input_img_resize=cv2.resize(input_img,(224,224))
        img_data_list.append(input_img_resize)
        
img_data = np.array(img_data_list)
img_data = img_data.astype('float32')
img_data = img_data/255
img_data.shape

Loaded the images of dataset-FEAR

Loaded the images of dataset-SURPRISE

Loaded the images of dataset-NEUTRAL

Loaded the images of dataset-DISGUST

Loaded the images of dataset-SAD

Loaded the images of dataset-HAPPY

Loaded the images of dataset-ANGRY



(213, 224, 224, 3)

In [28]:
num_classes = 7#生气、厌恶、恐惧、快乐、没啥表情、伤心、吃惊

num_of_samples = img_data.shape[0]
labels = np.ones((num_of_samples,),dtype='int64')

labels[0:32]=0 #32
labels[32:62]=1 #30
labels[62:92]=2 #30
labels[92:121]=3 #29
labels[121:152]=4 #31
labels[152:183]=5 #31
labels[183:]=6 #30

names = ['FEAR','SURPRISE','NEUTRAL','DISGUST','SAD','HAPPY','ANGRY']

def getLabel(id):
    return ['FEAR','SURPRISE','NEUTRAL','DISGUST','SAD','HAPPY','ANGRY'][id]
Y=labels
Y.shape

(213,)

In [29]:
features=get_features(img_data,2)

In [9]:
from sklearn.model_selection import cross_val_score

In [30]:
from sklearn.neighbors import KNeighborsClassifier as KNN
knn = KNN(n_neighbors=1)
scores = cross_val_score(knn, features, labels, cv=10)
print(scores.mean())

0.4216450216450216


In [18]:
# X=model.predict(img_data[0:1,:,:,:])
# X[12].shape

In [19]:
from sklearn.svm import SVC
clf = SVC()
scores = cross_val_score(clf, features, labels, cv=10)

/root/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/root/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/root/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/root/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureW

In [20]:
print(scores.mean())

0.3652922077922077


In [21]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
scores = cross_val_score(rf, features, labels, cv=10)
print(scores.mean())

/root/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/root/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/root/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/root/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/root/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:2

0.4133333333333333
